In [1]:
pip install neo4j


  Created wheel for neo4j: filename=neo4j-4.4.3-py3-none-any.whl size=116068 sha256=0138860db9f6fa1ba9b62191e1def0cd8bc56af41c71d141713bf1c7d6c5f997
  Stored in directory: c:\users\18483\appdata\local\pip\cache\wheels\a0\8c\08\e5396fee5c4d6c2e7901c049aad7aec428eafe2d752565019c
Successfully built neo4j


In [9]:
from neo4j import GraphDatabase
from neo4j import Neo4jConnection

ImportError: cannot import name 'Neo4jConnection' from 'neo4j' (C:\Users\18483\anaconda3\lib\site-packages\neo4j\__init__.py)

In [4]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [5]:
conn = Neo4jConnection(uri="bolt:/localhost:7474", user="neo4j", pwd="19882001")

In [6]:
conn.query("MATCH (n) RETURN count(n)")

[<Record count(n)=215672>]

In [ ]:
from pandas import DataFrame
query_string = '''
MATCH (p:Paper)
RETURN 
'''
dtf_data = DataFrame([dict(_) for _ in conn.query(query_string, db='coradb')])
dtf_data.sample(10)

In [27]:
conn.query("MATCH (n) DETACH DELETE n") ## удалить ноды

Query failed: {code: Neo.TransientError.General.OutOfMemoryError} {message: There is not enough memory to perform the current task. Please try increasing 'dbms.memory.heap.max_size' in the neo4j configuration (normally in 'conf/neo4j.conf' or, if you are using Neo4j Desktop, found through the user interface) or if you are running an embedded installation increase the heap by using '-Xmx' command line flag, and then restart the database.}


In [7]:
query_string = '''
CALL gds.pageRank.stream('neo4j', {
  writeProperty: 'pagerank'
})
YIELD nodePropertiesWritten, ranIterations
'''
conn.query(query_string, db='neo4j')

query_string = '''
CALL gds.betweenness.stream('neo4j', { 
  writeProperty: 'betweenness' })н  
YIELD minimumScore, maximumScore, scoreSum, nodePropertiesWritten
'''
conn.query(query_string, db='coradb')

Query failed: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `gds.pageRank.stream` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}
Query failed: {code: Neo.ClientError.Database.DatabaseNotFound} {message: Database does not exist. Database name: 'coradb'.}
